# Implementazione della Rilevazione in Tempo Reale

## Settings
In questo script, implementiamo la logica per utilizzare il nostro modello addestrato su un flusso video live proveniente da una webcam. Il processo si articola in questi punti:
1.  **Caricamento dei Modelli**: Carichiamo sia il nostro modello di classificazione delle emozioni (`.h5` o `.keras`) sia il classificatore Haar di OpenCV per la rilevazione dei volti (`.xml`).
2.  **Acquisizione Video**: Avviamo la cattura del video dalla webcam.
3.  **Ciclo di Elaborazione**: In un ciclo continuo, leggiamo ogni fotogramma del video, rileviamo i volti, e per ogni volto trovato applichiamo il nostro modello per predirne l'emozione.
4.  **Visualizzazione**: Mostriamo a schermo il video con i riquadri attorno ai volti e l'etichetta dell'emozione predetta.

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# --- 1. Caricamento delle Risorse ---

# Caricamento del modello per il riconoscimento delle emozioni
model = tf.keras.models.load_model('frank_emotion_detector_model.keras')

# Caricamento del classificatore a cascata Haar per la rilevazione dei volti
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Definiamo le etichette delle emozioni in ordine, come nel training
emotion_labels = ["Rabbia", "Disgusto", "Paura", "Felicita'", "Neutralita'", "Tristezza", "Sorpresa"]


C:\Users\vitaa\anaconda3\envs\cv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\vitaa\anaconda3\envs\cv\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.5 when it was built against 1.14.6, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


## Avviamento Webcam e il Ciclo Principale

Ora inizializziamo la webcam e creiamo un ciclo while che continuerà ad essere eseguito finché non decideremo di chiudere il programma. Questo ciclo è il cuore della nostra applicazione in tempo reale.

Il seguente blocco di codice contiene il ciclo principale dell'applicazione. Per ogni fotogramma catturato dalla webcam:

1. Viene convertito in scala di grigi, formato ottimale per il classificatore Haar.
2. Il classificatore `detectMultiScale` rileva tutti i volti presenti, restituendo le loro coordinate.
3. Per ogni volto trovato (chiamato ROI - *Region of Interest*), eseguiamo la pre-elaborazione necessaria per il nostro modello (resize a 224x224, aggiunta della dimensione del batch) e lo passiamo al modello per ottenere una predizione.
4. Infine, disegniamo un rettangolo attorno al volto e scriviamo l'emozione predetta sul fotogramma originale, che viene poi mostrato a schermo.

In [2]:
# --- 2. Avvio della Webcam e Ciclo Principale ---

# Inizializza la cattura video dalla webcam predefinita (indice 0)
cap = cv2.VideoCapture(0)

while True:
    # Legge un singolo fotogramma (frame) dalla webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Converte il frame in scala di grigi per il face detector
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Rileva i volti nel frame in scala di grigi
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Itera su ogni volto rilevato
    for (x, y, w, h) in faces:
        # Estrae la regione di interesse (ROI - il volto) dal frame a colori
        face_roi = frame[y:y+h, x:x+w]

        # --- Pre-elaborazione del volto per il nostro modello ---
        # 1. Ridimensiona il volto a 224x224, come richiesto dal modello
        resized_face = cv2.resize(face_roi, (224, 224))
        # 2. Aggiunge una dimensione per il "batch" (il modello si aspetta un gruppo di immagini)
        input_image = np.expand_dims(resized_face, axis=0)

        # --- Predizione (Inferenza) ---
        # Usa il modello per predire l'emozione. La funzione predict è velocissima.
        prediction = model.predict(input_image)

        # Ottiene l'indice dell'emozione con la probabilità più alta
        predicted_class_index = np.argmax(prediction)

        # Ottiene l'etichetta testuale corrispondente
        predicted_emotion = emotion_labels[predicted_class_index]

        # --- Visualizzazione ---
        # Disegna un rettangolo verde attorno al volto rilevato
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # Scrive l'etichetta dell'emozione predetta sopra il rettangolo
        cv2.putText(frame, predicted_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Mostra il frame finale con le rilevazioni in una finestra
    cv2.imshow('Riconoscimento Emozioni', frame)

    # Interrompe il ciclo se viene premuto il tasto 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 3. Rilascio delle Risorse ---
# Rilascia la webcam e chiude tutte le finestre di OpenCV
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 166ms/step
